# Laboratorio 2

Modelación y Simulación

Edwin Ortega 22305 - Esteban Zambrano 22119 - Juan Pablo Solis 22102

Ejercicios en donde se necesita usar Julia

In [1]:
using JuMP
using HiGHS
using Ipopt
using Optimization

### Ejercicio 1: Problema de transporte de gasolina

#### Inciso a

In [4]:
# Conjuntos
ref = ["R1","R2","R3"]
area = ["A1","A2","A3"]

# Oferta (millones de galones/día)
s = Dict("R1"=>6, "R2"=>5, "R3"=>8)

# Demanda (millones de galones/día)
d = Dict("A1"=>4, "A2"=>8, "A3"=>7)

# Distancias (km) – rutas existentes
dist = Dict(
    ("R1","A1") => 120, ("R1","A2") => 180,
    ("R2","A1") => 300, ("R2","A2") => 100, ("R2","A3") => 80,
    ("R3","A1") => 200, ("R3","A2") => 250, ("R3","A3") => 120,
)

# Costo por millón de galones
cost(i,j) = 100 * dist[(i,j)]   # 0.10 USD / 1000 gal × 1000

# Modelo
model = Model(HiGHS.Optimizer)
@variable(model, x[ref, area] >= 0)

# Eliminar rutas inexistentes
for i in ref, j in area
    if !haskey(dist, (i,j))
        fix(x[i,j], 0; force = true)
    end
end

# Función objetivo
@objective(model, Min, sum(cost(i,j)*x[i,j] for (i,j) in keys(dist)))

# Restricciones
@constraint(model, [i in ref], sum(x[i,j] for j in area) <= s[i])  # oferta
@constraint(model, [j in area], sum(x[i,j] for i in ref) == d[j])  # demanda

optimize!(model)

println("Costo mínimo diario: \$", objective_value(model))
for i in ref, j in area
    if value(x[i,j]) > 1e-6
        println("$i → $j : $(round(value(x[i,j]), digits=3)) Mgal")
    end
end


Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 9 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+03, 3e+04]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 8e+00]
Presolving model
6 rows, 8 cols, 16 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
5 rows, 7 cols, 14 nonzeros  0s
Presolve : Reductions: rows 5(-1); columns 7(-2); elements 14(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          4     2.4300000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 4
Objective value     :  2.4300000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.02
Co

#### Inciso b

In [10]:
routes = [
    ("R1","A1", 4, cost("R1","A1")),
    ("R1","A2", 2, cost("R1","A2")),
    ("R2","A2", 5, cost("R2","A2")),
    ("R3","A2", 1, cost("R3","A2")),
    ("R3","A3", 7, cost("R3","A3")),
]

rows = [
    (Origen=o, Destino=d, Mgal=vol,
     USD_Mgal=unit, Subtotal_USD=vol*unit)
    for (o,d,vol,unit) in routes
]

pretty_table(rows; formatters = ft_printf("%.0f", 3:5))
println("\nCosto total: \$", objective_value(model))

┌────────┬─────────┬───────┬──────────┬──────────────┐
│ Origen │ Destino │  Mgal │ USD_Mgal │ Subtotal_USD │
│ String │  String │ Int64 │    Int64 │        Int64 │
├────────┼─────────┼───────┼──────────┼──────────────┤
│     R1 │      A1 │     4 │    12000 │        48000 │
│     R1 │      A2 │     2 │    18000 │        36000 │
│     R2 │      A2 │     5 │    10000 │        50000 │
│     R3 │      A2 │     1 │    25000 │        25000 │
│     R3 │      A3 │     7 │    12000 │        84000 │
└────────┴─────────┴───────┴──────────┴──────────────┘

Costo total: $243000.0


#### Inciso c

In [11]:
# --- Datos ---
ref = ["R1","R2","R3"]
area = ["A1","A2","A3"]

s  = Dict("R1"=>6,"R2"=>5,"R3"=>8)               # oferta
d2 = Dict("A1"=>4,"A2"=>8,"A3"=>4)               # nueva demanda

dist = Dict(
    ("R1","A1") => 120, ("R1","A2") => 180,
    ("R2","A1") => 300, ("R2","A2") => 100, ("R2","A3") => 80,
    ("R3","A1") => 200, ("R3","A2") => 250, ("R3","A3") => 120,
)

cost(i,j) = 100 * dist[(i,j)]                    # USD / Mgal

overflow_cost = Dict("R1"=>15_000,
                     "R2"=>22_000,
                     "R3"=>0)                    # USD / Mgal

# --- Modelo ---
model_c = Model(HiGHS.Optimizer)

@variable(model_c, x[ref, area] >= 0)            # envíos por oleoducto
@variable(model_c, y[ref]       >= 0)            # excedente

# Desactivar rutas inexistentes
for i in ref, j in area
    if !haskey(dist, (i,j))
        fix(x[i,j], 0; force = true)
    end
end

@objective(model_c, Min,
    sum(cost(i,j) * x[i,j] for i in ref, j in area if haskey(dist,(i,j))) +
    sum(overflow_cost[i] * y[i] for i in ref)
)

@constraint(model_c, [i in ref], sum(x[i,j] for j in area) + y[i] == s[i])  # oferta exacta
@constraint(model_c, [j in area], sum(x[i,j] for i in ref) == d2[j])        # demanda exacta

optimize!(model_c)

println("Estado: ", termination_status(model_c))
println("Costo mínimo diario: \$", objective_value(model_c))

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 6 rows; 12 cols; 21 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+03, 3e+04]
  Bound  [0e+00, 0e+00]
  RHS    [4e+00, 8e+00]
Presolving model
6 rows, 11 cols, 19 nonzeros  0s
Dependent equations search running on 5 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
5 rows, 10 cols, 17 nonzeros  0s
Presolve : Reductions: rows 5(-1); columns 10(-2); elements 17(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          5     2.0700000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 5
Objective value     :  2.0700000000e+05
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.0

In [12]:
# --- rutas activas ---
routes = []
for i in ref, j in area
    q = value(x[i,j])
    if q > 1e-6
        push!(routes, (Origen=i, Destino=j,
                       Mgal=q,
                       USD_Mgal=cost(i,j),
                       Subtotal_USD=q*cost(i,j)))
    end
end

pretty_table(routes; formatters=ft_printf("%.0f", 3:5))
println("\nExcedentes:")
for i in ref
    println("  $(i): ", round(value(y[i]),digits=3), " Mgal  (costo USD/Mgal = ", overflow_cost[i], ")")
end
println("\nCosto total: \$", objective_value(model_c))


┌───────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                Col. 1 │
├───────────────────────────────────────────────────────────────────────────────────────┤
│ (Origen = "R1", Destino = "A1", Mgal = 4.0, USD_Mgal = 12000, Subtotal_USD = 48000.0) │
│ (Origen = "R1", Destino = "A2", Mgal = 2.0, USD_Mgal = 18000, Subtotal_USD = 36000.0) │
│ (Origen = "R2", Destino = "A2", Mgal = 5.0, USD_Mgal = 10000, Subtotal_USD = 50000.0) │
│ (Origen = "R3", Destino = "A2", Mgal = 1.0, USD_Mgal = 25000, Subtotal_USD = 25000.0) │
│ (Origen = "R3", Destino = "A3", Mgal = 4.0, USD_Mgal = 12000, Subtotal_USD = 48000.0) │
└───────────────────────────────────────────────────────────────────────────────────────┘

Excedentes:
  R1: 0.0 Mgal  (costo USD/Mgal = 15000)
  R2: 0.0 Mgal  (costo USD/Mgal = 22000)
  R3: 3.0 Mgal  (costo USD/Mgal = 0)

Costo total: $207000.0
